## Model Drift Detection Preparation

This notebook is used to create inference data for the drift detection notebooks.  The plan is to run these an hour or a day before so the workshop participants can use them for their training.

This notebook will have the bare minimum necessary for the training.  The rest will be part of the N4_drift_detection.ipynb notebook to actually demonstrate using assays for drift detection.

## Steps

* Load the workspace, pipeline, and model versions.
* Perform sample inferences to:
  * Set the baseline
  * Perform "normal" inferences.
  * Perform inferences that should trigger alerts.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time


workspace_name = 'workshop-workspace-john-05'
main_pipeline_name = 'houseprice-estimator'
model_name_control = 'house-price-prime'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Retrieve Workspace, Pipeline, and Models

Retrieve the workspace, pipeline and model from notebook N1_deploy_a_model.ipynb.

In [4]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-05"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'house-price-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'houseprice-estimator'

pipeline = wl.get_pipeline(pipeline_name)

display(workspace)
display(prime_model_version)
display(pipeline)


{'name': 'workshop-workspace-john-05', 'id': 11, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-04-22T20:33:29.332164+00:00', 'models': [{'name': 'house-price-prime', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 47, 17, 816549, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 34, 37, 434083, tzinfo=tzutc())}, {'name': 'house-price-rf-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 36, 195788, tzinfo=tzutc())}, {'name': 'house-price-gbr-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 4, 22, 20, 53, 39, 543410, tzinfo=tzutc())}], 'pipelines': [{'name': 'houseprice-estimator', 'create_time': datetime.datetime(2024, 4, 22, 20, 34, 49, 55

Name,house-price-prime
Version,be93bbfc-4f0c-444d-aa89-5f15dec01d83
File Name,xgb_model.onnx
SHA,31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Apr 20:47:17


name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-22 21:06:21.434281+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


### Deploy Pipeline

Deploy the pipeline with the model version.

In [5]:
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 15:16:55.785413+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


### Generate Sample Data

Before creating the assays, we must generate data for the assays to build from.

For this example, we will:

* Perform sample inferences based on lower priced houses and use that as our baseline.
* Generate inferences from specific set of high priced houses create inference outputs that will be outside the baseline.  This is used in later steps to demonstrate baseline comparison against assay analyses.

#### Inference Results History Generation

To start the demonstration, we'll create a baseline of values from houses with small estimated prices and set that as our baseline.

We will save the beginning and end periods of our baseline data to the variables `assay_baseline_start` and `assay_baseline_end`.

In [6]:
small_houses_inputs = pd.read_json('../data/lowprice.df.json')
baseline_size = 500

# Where the baseline data will start
assay_baseline_start = datetime.datetime.now()

# These inputs will be random samples of small priced houses.  Around 30,000 is a good number
small_houses = small_houses_inputs.sample(baseline_size, replace=True).reset_index(drop=True)

# Wait 60 seconds to set this data apart from the rest
time.sleep(60)
small_results = pipeline.infer(small_houses)

# Set the baseline end

assay_baseline_end = datetime.datetime.now()

#### Generate Numpy Baseline Values

This process generates a numpy array of the inference results used as baseline data in later steps.

In [7]:
# get the numpy values

# set the results to a non-array value
small_results_baseline_df = small_results.copy()
small_results_baseline_df['variable']=small_results['out.variable'].map(lambda x: x[0])
small_results_baseline_df

# set the numpy array
small_results_baseline = small_results_baseline_df['variable'].to_numpy()

#### Assay Test Data

The following will generate inference data for us to test against the assay baseline.  For this, we will add in house data that generate higher house prices than the baseline data we used earlier.

This process should take 6 minutes to generate the historical data we'll later use in our assays.  We store the DateTime `assay_window_start` to determine where to start out assay analyses.

In [8]:
# Get a spread of house values

# # Set the start for our assay window period.
assay_window_start = datetime.datetime.now()

time.sleep(65)
inference_size = 1000

# And a spread of large house values

small_houses_inputs = pd.read_json('../data/lowprice.df.json', orient="records")
small_houses = small_houses_inputs.sample(inference_size, replace=True).reset_index(drop=True)

pipeline.infer(small_houses)

time.sleep(65)

In [9]:
# Get a spread of large house values

time.sleep(65)
inference_size = 1000

# And a spread of large house values

big_houses_inputs = pd.read_json('../data/highprice.df.json', orient="records")
big_houses = big_houses_inputs.sample(inference_size, replace=True).reset_index(drop=True)

pipeline.infer(big_houses)

time.sleep(65)

### Undeploy Main Pipeline

With the examples and tutorial complete, we will undeploy the main pipeline and return the resources back to the Wallaroo instance.

In [11]:
pipeline.undeploy()

name,houseprice-estimator
created,2024-04-22 20:34:49.559433+00:00
last_updated,2024-04-24 15:16:55.785413+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"3eedd163-bd54-4bde-943e-3f8b32a1f47c, 957ea8c5-d5c0-4629-82fb-fde09cf06958, e6fc1ebc-1a8f-48d2-9709-41c6820d6158, d0517194-4275-4684-860b-b35d64efba0a, 95cf17b2-19b1-40b6-82a5-d50b7f78094d, fad60d26-5f2f-467b-82ab-b97e60cfae2a, 71d8ae65-6b5e-422e-9d91-90fed507f74a, d80ef023-a703-4822-910d-b84f8d20174d, b99c6d8c-1d57-49ce-9690-07fa8f6988db, fbdf00f2-104e-41c6-9df7-86127bd2322e, 1ba316a5-6523-47f1-9e44-ec4c30e5710a"
steps,house-price-prime
published,False


### Store the Assay Values

We will store the following into a location configuration file:

* `small_results_baseline`:  Used to create the baseline from the numpy values from sample inferences.
* `assay_baseline_start`: When to start the baseline from the inference history.
* `assay_baseline_end`: When to end the baseline from the inference history.
* `assay_window_start`: When to start the assay window period for assay samples.

In [12]:
# skip this step if the file is already there

import numpy

numpy.save('./small_results_baseline.npy', small_results_baseline)

In [13]:
baseline_numpy = numpy.load('./small_results_baseline.npy')

In [14]:
with open('./assay_baseline_start', 'w') as file:
    file.write(assay_baseline_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_start

datetime.datetime(2024, 4, 24, 9, 17, 21, 718515)

In [15]:
with open('./assay_baseline_end', 'w') as file:
    file.write(assay_baseline_end.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_baseline_end

datetime.datetime(2024, 4, 24, 9, 18, 23, 126952)

In [16]:
with open('./assay_window_start', 'w') as file:
    file.write(assay_window_start.strftime("%d-%b-%Y (%H:%M:%S.%f)"))
assay_window_start

datetime.datetime(2024, 4, 24, 9, 18, 23, 154891)

In [17]:
# read the assay baseline start datetime

with open('./assay_baseline_start', 'r') as file:
    assay_baseline_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_start_test

datetime.datetime(2024, 4, 24, 9, 17, 21, 718515)

In [18]:
# read the assay baseline end datetime

with open('./assay_baseline_end', 'r') as file:
    assay_baseline_end_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_baseline_end_test

datetime.datetime(2024, 4, 24, 9, 18, 23, 126952)

In [19]:
# read the assay window start datetime

with open('./assay_window_start', 'r') as file:
    assay_window_start_test = datetime.datetime.strptime(file.read(), "%d-%b-%Y (%H:%M:%S.%f)")
assay_window_start_test

datetime.datetime(2024, 4, 24, 9, 18, 23, 154891)